# Bak naar image naar kaart naar elasticsearch

## Bak naar image naar kaart naar elasticsearch

Converteer excel-files met beschrijving van de kaartenbakken naar imagename en expand images naar kaarten per (emigranten)
unit. Indexeer vervolgens met de unit als parent naar elasticsearch

## excel naar csv

In [2]:
import csv
from operator import itemgetter
import xlrd
import os
import string
import re

# read excel files and convert only the first worksheet to csv (adapted from example on the Internet)

def csv_from_excel(excel_file, outdir):
    workbook = xlrd.open_workbook(excel_file)
    all_worksheets = workbook.sheet_names()
    worksheet_name = all_worksheets[0]
    worksheet = workbook.sheet_by_name(worksheet_name)
    nm = os.path.basename(excel_file)
    nm = os.path.splitext(nm)[0]
    out = os.path.join(outdir, nm + '_' + worksheet_name +'.csv')
    print out
    try:
        os.makedirs(outdir)
    except OSError:
        pass
    your_csv_file = open(out, 'wb')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)
    for rownum in xrange(worksheet.nrows):
        wr.writerow([unicode(entry).encode("utf-8") for entry in worksheet.row_values(rownum)])
    print "converted %s" % excel_file
    your_csv_file.close()

# help function to get drawer number from drawer description. Later on replaced with bak from imagename
def bak2nr(flin='/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/bakken.csv'):
    flin = open(flin)
    r = csv.DictReader(flin)
    lst = {row['archieflink']: int(row['nr']) for row in r}
    return lst

# expand number of images per unit to cover alle images in  drawer (as far as described in excel file)
def relpers2scan(fl):
    """link person identifier to scans numbers
    example of flin: /home/rik/Dropbox/Link to emigrantenkaarten/bak_nw_south_wales_bew.csv
    """
    logger = logging.getLogger(__name__)
    flin = open(fl)
    logger.info("converting %s" % flin)
    reader = csv.DictReader(flin)
    baknrs = bak2nr()
    scans = {}
    #files are supposed to have at least the following fields
    if not set(['imagenr', 'prs_uuid','aant','kaartenbak']) < set(reader.fieldnames): 
        logging.debug("FILE DOES NOT HAVE THE RIGHT FIELDS") # mark for postprocessing
        raise ValueError
               
    else:
        archieflink = re.search('bak([0-9]+)', fl).groups()[0]
        for i in reader:
            if i['imagenr'] != '' and i['imagenr'][0] in string.digits:
#                if i['archieflink'] in baknrs.keys():
#                    nr = baknrs[i['archieflink']]
                    
                i['archieflink'] = archieflink
                scans[int(i['imagenr'].split('.')[0])] = [i['prs_uuid'], i['aant'],i['archieflink']]
#        scans.sort(key=itemgetter(0))
        lst = scans.keys()
        mn = min(lst)
        mx = max(lst)
        scanlist = range(mn,mx + 2)
        nwscans = []
        curscan = scans[mn]
        for i in scanlist:
            if i in lst:
                curscan = scans[i]    
            result = [i]
            result.extend(curscan)
            nwscans.append(result)               
        return nwscans    

#convert original csv to csv with limited files for all images in drawer
def kaart2image(f, outdir):
    fl = os.path.basename(f)
    fl = os.path.splitext(fl)[0]
    flin = fl + '.csv'
    fli = os.path.join(outdir, flin)
    logger = logging.getLogger(__name__)
    logger.info("infile: %s" % f)
    try:
        nf = relpers2scan(f)
        flout = os.path.join(outdir, fl+'_out.csv')
        flt = open(flout, 'wb')
        w = csv.writer(flt)
        w.writerow(['image', 'kaart', 'aantekening', 'bak'])
        w.writerows(nf)
        flt.close()
        logger.info("written: %s" % flt)
    except (ValueError, IndexError, TypeError):
        pass # dangerous!

#actual conversion. First step - convert excel to csv
def convert(indir):
    import glob, os, logging
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename='/Users/rikhoekstra/Documents/bak2image.log', mode='w')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.DEBUG)
    
    fls = glob.glob(indir + '*.xlsx')
    outdir = os.path.join(indir, 'csv')
    logging.info(outdir)
    for fl in fls:
        csv_from_excel(fl, outdir)
    return 'done'


In [ ]:
#let's do this on the input directory given below. The function will emit all converted file names
indir = "/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/"
convert(indir)

In [ ]:
# and perform step 2 to expand images. 
# Remember to check the logs for files with faulty field names. These must be remedied by hand

import glob, os, logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='/Users/rikhoekstra/Documents/bak2image.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

outdir = os.path.join(indir, 'csv')
nfls = glob.glob(os.path.join(outdir, '*.csv'))
for f in nfls:
        kaart2image(f, outdir)

## Now import the image files into Elastic Search

In [ ]:
# Run this cell to import the necessary libraries and initiate an Elasticsearch object.
import unicodecsv
import os
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError
import datetime

import json
from IPython.core.display import display, HTML
import re

es = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True) 

In [ ]:
# create mapping in index (the index should already exist)
index = 'test_migrant'

request_body = {   
            "image": {
                    "_parent": {"type": "unit"},
                    "properties":{
                        'image_id': {"type":"string"},
                        'aantekening':{"type":"text"},
                        'edges': {"type":"integer"},
                        'ocr': {u'type': u'text'},
                        'locr': {u'type':u'integer'}
                    }
                },
    }

INDEX_NAME=index
es.indices.put_mapping(index = index, doc_type="image",  body = request_body)
    
# check mapping
es.indices.get_mapping(index = index)

### and now for some name manipulation

In [ ]:
# get edges and ocr for images helping functions
edgefile = "/Users/rikhoekstra/surfdrive/emigratie/alledges.csv"
ocrdir = "/Users/rikhoekstra/surfdrive/Shared/Database_Migrant/OCR_bakken/media/rutger/encrypted/emigratieregistratie"
flin = open(edgefile)
r = csv.DictReader(flin)
rows = [row for row in r]
def getedges(im, rows=rows, ocrdir=ocrdir):
    ocrdir = ocrdir
    result = {}
    edge = [d['edges'] for d in rows if d['file'] == im][0]
    result['edge'] = edge
    bak = "{:02d}".format(int(im.split('_')[2]))
    imname = os.path.join(ocrdir, bak, im+'.txt')
    flin = open(imname)
    ocr = flin.read()
    flin.close()
    result['ocr'] = ocr
    result['locr'] = len(ocr)
    return result
    
# test (but don't expect too much from the ocr)
getedges("NL-HaNA_2.05.159_8_0008.jpg")

In [ ]:

url = "https://migrantcards.huygens.knaw.nl/cards"

urltemplate = "{url}/{map:02d}/NL-HaNA_2.05.159_{map}_{imnr:04d}.jpg"
imagetemplate = "NL-HaNA_2.05.159_{map}_{imnr:04d}.jpg"

#test 
print(urltemplate.format(url=url, map=8, imnr=8))
print(imagetemplate.format(map=8, imnr=8))


### get edges and ocr from other files

In [ ]:
from unicodecsv import DictReader
imfls = [fl for fl in imfls if fl not in outfls]
faulty_items = []
for fn in imfls:
    try:
        flin = open(fn)
        r = DictReader(flin)
        ims = [row for row in r]
        for row in ims:
            row['parent'] = row['kaart']
            row['aantekening'] = row['aantekening']
            edocr = getedges(imagetemplate.format( map=int(row['bak']), imnr=int(row["image"])))
            row['id'] = "{map}_{imnr:04d}".format(map=row['bak'], imnr=int(row['image']))
            row['url'] = urltemplate.format(url=url, map=int(row['bak']), imnr=int(row["image"]))
            row['edges'] = int(edocr['edge'])
            row['locr'] = int(edocr['locr'])
            row['ocr'] = dammit(edocr['ocr']).unicode_markup
        
    except IndexError, KeyError:
        faulty_items.append(row)
    data_dict = ims
    bulk_data = []
    for item in data_dict:
        try:
            bulk_data.append({'index':{'_type':'image', 
                                       '_index':index,
                                       'parent': item['parent'],
                                       '_id':'%s' % item.get('id')}})
            bulk_data.append(item)
        except KeyError:
            print fn
            print item
    


In [ ]:
imfls = glob.glob(os.path.join(outdir, '*_out.csv'))
outfls = ["/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak10_Melbourne_klaar_Sheet1_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak11_Melbourne_klaar_Sheet1_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak12_Melbourne_klaar_Sheet1_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak13_Melbourne_klaar_Sheet1_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak1_Brisbane_klaar_Blad1_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak27_nw_south_wales_klaar_bak_nw_south_wales_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak28_Queensland_klaar_Bak_28_Queensland_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak29_Sydney_klaar_bak_29_Sydney_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak2_Brisbane_klaar_Blad1_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak30_Sydney_klaar_Bak_30_Sydney_out.csv handled",
"/Users/rikhoekstra/surfdrive/emigratie/emigrantenkaarten/database/bakken_geordend/csv/bak31_Sydney_klaar_Blad1_out.csv handled"]

In [ ]:
from unicodecsv import DictReader
imfls = [fl for fl in imfls if fl not in outfls]
faulty_items = []
for fn in imfls:
    try:
        flin = open(fn)
        r = DictReader(flin)
        ims = [row for row in r]
        for row in ims:
            row['parent'] = row['kaart']
            row['aantekening'] = row['aantekening']
            edocr = getedges(imagetemplate.format( map=int(row['bak']), imnr=int(row["image"])))
            row['id'] = "{map}_{imnr:04d}".format(map=row['bak'], imnr=int(row['image']))
            row['url'] = urltemplate.format(url=url, map=int(row['bak']), imnr=int(row["image"]))
            row['edges'] = int(edocr['edge'])
            row['locr'] = int(edocr['locr'])
            row['ocr'] = dammit(edocr['ocr']).unicode_markup
        
    except IndexError, KeyError:
        faulty_items.append(row)
    data_dict = ims
    bulk_data = []
    for item in data_dict:
        try:
            bulk_data.append({'index':{'_type':'image', 
                                       '_index':index,
                                       'parent': item['parent'],
                                       '_id':'%s' % item['id']}})
            bulk_data.append(item)
        except KeyError:
            print fn
            print item

    index = "migrants"
    es.bulk(index=index, body=bulk_data, refresh=True)
    outfls.append(fn)


In [ ]:
outfls

In [ ]:
d['errors']

In [1]:
bulk_data_file = open()

NameError: name 'os' is not defined